# 1 导入库

In [1]:
import pandas as pd 
import numpy as np 
from nltk import word_tokenize 
import nltk
nltk.download('punkt')
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

from tensorflow.keras.layers import Dense,Input,Dropout,Embedding,LSTM,Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split

import json

import time # 记录模型训练时间


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 2 设置神经网络参数

为了方便统一设置，我们在这里指定神经网络训练时所需要的参数，在后面的代码中可以直接引用。

In [21]:
# 批次大小
batch_size = 128

# 训练周期
epochs = 3

# 词向量长度
embedding_dims = 128

# cell数量
lstm_cell = 64

# 3 数据加载


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
import os
#os.chdir("/content/drive/Shared drives/项目名")
os.chdir("/content/gdrive/MyDrive/ai_lab_final")

In [5]:
data = pd.read_csv('train.csv')

## 3.1 统计样本数量

In [6]:
print('样本总数：%d' % data.shape[0])

# 计算正样本数量
poslen = sum(data['gold_label']==1)

# 计算负样本数量
neglen = sum(data['gold_label']==0)
print('正样本数量：', poslen)
print('负样本数量：', neglen)

样本总数：13000
正样本数量： 1454
负样本数量： 11546


# 4 文本分词

## 4.1 调用nltk包中的分词函数

In [7]:
#定义分词函数，对传入的x进行分词
cw = lambda x: list(word_tokenize(x))

# apply传入一个函数，把cw函数应用到data['sentence']的每一行
# 把分词后的结果保存到data['words']中
data['words'] = data['sentence'].apply(cw)

# 5 文本数据预处理


## 5.1 统计词汇最大长度

In [8]:
max_length = max([len(x) for x in data['words']])

## 5.2 数据格式转换

把 `data['words']`  中所有的 `list` 都变成字符串格式：

In [9]:
texts = [' '.join(x) for x in data['words']]

## 5.3 `Tokenizer` 获取词向量

In [10]:
# 实例化Tokenizer，设置字典中最大词汇数为30000
# Tokenizer会自动过滤掉一些符号比如：!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n
tokenizer = Tokenizer(num_words=30000)

# 传入我们的训练数据，建立词典，词的编号根据词频设定，频率越大，编号越小，
tokenizer.fit_on_texts(texts) 

# 把词转换为编号，编号大于30000的词会被过滤掉
sequences = tokenizer.texts_to_sequences(texts) 

## 5.4 序列填充

- 把序列设定为 `max_length` 的长度，超过 `max_length` 的部分舍弃，不到 `max_length` 则补 `0`
- `padding='pre'` 在句子前面进行填充，`padding='post'` 在句子后面进行填充，本实验选择句前填充

In [11]:
X = pad_sequences(sequences, maxlen=max_length, padding='pre')

## 5.5 保存 Tokenizer

将 `tokenizer` 保存为 json 文件：

In [12]:
# 把token_config保存到json文件中，模型预测阶段可以使用
file = open('./model/token_config_lstm.json', 'w', encoding = "ISO-8859-1")

# 把tokenizer变成json数据
token_config = tokenizer.to_json()

# 保存json数据
json.dump(token_config, file)

## 5.6 定义标签

将标签定义为独热编码，其中 `01` 为正样本 (积极)，`10` 为负样本 (消极)：

In [13]:
positive_labels = [[0, 1] for _ in range(poslen)]
negative_labels = [[1, 0] for _ in range(neglen)]

## 5.7 合并标签

将正样本 (积极) 标签和负样本 (消极) 标签进行合并用于训练模型：

In [14]:
Y = np.array(positive_labels + negative_labels)

## 5.8 切分数据

调用 `train_test_split` 对数据进行切分，其中测试集占 `0.2`，即训练集与测试集比例为 8:2

In [15]:
x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2)

# 6 模型定义

## 自定义f1score计算

In [16]:
class Metrics(Callback):
  def __init__(self, val_data, val_label):
      super(Callback, self).__init__()
      self.val_data = val_data
      self.val_label = val_label
  def on_train_begin(self, logs={}):
    self.val_f1s = []
  def on_epoch_end(self, epoch, logs={}):
    val_predict=(np.asarray(self.model.predict(self.val_data))).round()
    val_targ = self.val_label
    _val_f1 = f1_score(val_targ, val_predict, average='micro')
    self.val_f1s.append(_val_f1)
    print("— val_f1: %f " % _val_f1)
    return

## 自定义学习率

In [17]:
import keras.backend as K
from keras.callbacks import LearningRateScheduler
 
def scheduler(epoch):
    # 每隔1个epoch，学习率减小为原来的1/2
    if epoch % 3 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.5)
        print("lr changed to {}".format(lr * 0.5))
    return K.get_value(model.optimizer.lr)

## 模型定义

In [18]:
# 定义模型输入，shape-(batch, 202)
sequence_input = Input(shape=(max_length,))

# Embedding层，30000表示30000个词，每个词对应的向量为128维
embedding_layer = Embedding(input_dim=30000, output_dim=embedding_dims)

# embedded_sequences的shape-(batch, 202, 128)
embedded_sequences = embedding_layer(sequence_input)

# 双向LSTM
x = Bidirectional(LSTM(lstm_cell))(embedded_sequences)

# 全连接层
x = Dense(128, activation='relu')(x)

# Dropout层
x = Dropout(0.5)(x)

# 输出层
preds = Dense(2, activation='softmax')(x)

# 定义模型
model = Model(sequence_input, preds)

神经网络模型架构如下所示：

![Alt text](./img/lstm_model.h5.svg)

## 模型编译

In [19]:
#自定义学习率
reduce_lr = LearningRateScheduler(scheduler)
# 自定义f1 score
metrics = Metrics(x_test, y_test)
# 定义代价函数，优化器
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

## 模型训练

In [22]:
start = time.time() # 记录训练开始时间

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[metrics, reduce_lr],
          validation_data=(x_test, y_test))
end = time.time() # 记录训练结束时间
print('训练耗时：',end - start, '秒')

Epoch 1/3
82/82 [==============================] - 191s 2s/step - loss: 0.3612 - acc: 0.8868 - val_loss: 0.3406 - val_acc: 0.8935 - lr: 0.0010
Epoch 2/3
82/82 [==============================] - 190s 2s/step - loss: 0.3081 - acc: 0.8892 - val_loss: 0.3985 - val_acc: 0.8904 - lr: 0.0010
Epoch 3/3
82/82 [==============================] - 189s 2s/step - loss: 0.1502 - acc: 0.9477 - val_loss: 0.5272 - val_acc: 0.7996 - lr: 0.0010
训练耗时： 621.9807171821594 秒


## 模型保存

训练完一个模型后，为了以后重复使用，需要对模型的结果进行保存。用 Tensorflow 去实现神经网络，所要保存的就是神经网络中的各项权重值。

In [ ]:
model.save('./model/lstm_model.h5')